# Evaluation Notebook

This notebook retrieves tasks from Notion that match debug tasks in an eval folder, and prepares them for comparison and evaluation.

In [ ]:
# Imports
import json
import re
from pathlib import Path
from typing import Optional
from notion_client import Client

from inbox_agent.config import settings
from inbox_agent.notion import query_pages_filtered, get_block_plain_text, extract_property_value
from inbox_agent.pydantic_models import NotionTask, AIUseStatus

print("✓ Imports loaded")

In [ ]:
# ============ CONFIGURATION ============
# Control which eval folder to use
EVAL_FOLDER_NAME = "eval_1"  # Change this to use different eval folders (e.g., "eval_2", "eval_3")

# Derived paths
EVAL_FOLDER_PATH = settings.LOGS_DIR / EVAL_FOLDER_NAME
NOTION_TASKS_FILE = EVAL_FOLDER_PATH / "notion_tasks.json"

print(f"✓ Eval folder: {EVAL_FOLDER_PATH}")
print(f"✓ Notion tasks will be saved to: {NOTION_TASKS_FILE}")

In [ ]:
# ============ STEP 1: Extract Debug Task Information ============

def get_debug_task_files(eval_folder: Path) -> list[Path]:
    """Get all .md debug task files from eval folder."""
    return [f for f in eval_folder.glob("*.md") if f.is_file()]

def extract_task_model_from_debug_file(file_path: Path) -> dict:
    """
    Extract the Task Model JSON from a debug task markdown file.
    
    Returns:
        Dict with task model data, or None if not found
    """
    content = file_path.read_text(encoding='utf-8')
    
    # Find Task Model section
    match = re.search(r'## Task Model\s+```json\s+(.*?)\s+```', content, re.DOTALL)
    if not match:
        return None
    
    try:
        task_data = json.loads(match.group(1))
        return task_data
    except json.JSONDecodeError as e:
        print(f"Warning: Could not parse Task Model JSON in {file_path.name}: {e}")
        return None

def get_debug_task_titles(eval_folder: Path) -> list[str]:
    """Extract all task titles from debug task files in eval folder."""
    debug_files = get_debug_task_files(eval_folder)
    titles = []
    
    for file in debug_files:
        task_model = extract_task_model_from_debug_file(file)
        if task_model and 'title' in task_model:
            titles.append(task_model['title'])
    
    return titles

# Test: Get debug task titles
debug_titles = get_debug_task_titles(EVAL_FOLDER_PATH)
print(f"✓ Found {len(debug_titles)} debug tasks")
print(f"Sample titles: {debug_titles[:3]}")

In [ ]:
# ============ STEP 2: Retrieve Tasks from Notion ============

def retrieve_tasks_from_notion(notion_client: Client, task_titles: list[str]) -> list[dict]:
    """
    Retrieve tasks from Notion based on titles using OR filter.
    
    Args:
        notion_client: Notion client instance
        task_titles: List of task titles to retrieve
    
    Returns:
        List of Notion page objects
    """
    if not task_titles:
        return []
    
    # Build OR filter for all titles
    filter_dict = {
        "or": [
            {"property": "Name", "title": {"equals": title}}
            for title in task_titles
        ]
    }
    
    # Properties to retrieve (relevant to NotionTask model + AIUsefulness, UseAIStatus)
    filter_properties = [
        "Name",           # title
        "Project",        # projects (relation)
        "Importance",     # importance
        "Urgency",        # urgency
        "Impact Score",   # impact
        "UseAIStatus",    # ai_use_status
        "AIUsefulness"    # additional evaluation property
    ]
    
    # Query Notion
    response = query_pages_filtered(
        notion_client,
        settings.NOTION_TASKS_DATA_SOURCE_ID,
        filter_dict=filter_dict,
        filter_properties=filter_properties
    )
    
    return response['results']

# Initialize Notion client and retrieve tasks
notion = Client(auth=settings.NOTION_TOKEN)
notion_tasks_raw = retrieve_tasks_from_notion(notion, debug_titles)

print(f"✓ Retrieved {len(notion_tasks_raw)} tasks from Notion")
if notion_tasks_raw:
    print(f"Sample task ID: {notion_tasks_raw[0]['id']}")

In [ ]:
# ============ STEP 3: Parse and Save Notion Tasks ============

def parse_notion_task(notion_page: dict) -> dict:
    """
    Parse a Notion page object into a simplified task dict.
    
    Returns:
        Dict with task properties extracted from Notion
    """
    props = notion_page.get('properties', {})
    
    # Extract title
    title = get_block_plain_text(notion_page)
    
    # Extract properties
    importance_select = props.get('Importance', {}).get('select')
    urgency_select = props.get('Urgency', {}).get('select')
    ai_status_select = props.get('UseAIStatus', {}).get('select')
    ai_usefulness_select = props.get('AIUsefulness', {}).get('select')
    
    # Extract project relations
    project_relations = props.get('Project', {}).get('relation', [])
    project_ids = [rel['id'] for rel in project_relations]
    
    return {
        'id': notion_page['id'],
        'title': title,
        'importance': int(importance_select['name']) if importance_select else None,
        'urgency': int(urgency_select['name']) if urgency_select else None,
        'impact': props.get('Impact Score', {}).get('number'),
        'ai_use_status': ai_status_select['name'] if ai_status_select else None,
        'ai_usefulness': ai_usefulness_select['name'] if ai_usefulness_select else None,
        'project_ids': project_ids,
        'created_time': notion_page.get('created_time'),
        'last_edited_time': notion_page.get('last_edited_time')
    }

def save_notion_tasks(tasks_raw: list[dict], output_file: Path):
    """Parse and save Notion tasks to JSON file."""
    parsed_tasks = [parse_notion_task(task) for task in tasks_raw]
    
    # Save as JSON
    output_file.parent.mkdir(parents=True, exist_ok=True)
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(parsed_tasks, f, indent=2, ensure_ascii=False)
    
    return parsed_tasks

# Parse and save
notion_tasks_parsed = save_notion_tasks(notion_tasks_raw, NOTION_TASKS_FILE)
print(f"✓ Saved {len(notion_tasks_parsed)} parsed Notion tasks to {NOTION_TASKS_FILE}")
print(f"✓ Sample task: {notion_tasks_parsed[0]['title'] if notion_tasks_parsed else 'None'}")

In [ ]:
# ============ STEP 4: Load and Pair Tasks ============

def load_notion_tasks(json_file: Path) -> list[dict]:
    """Load parsed Notion tasks from JSON file."""
    if not json_file.exists():
        return []
    
    with open(json_file, 'r', encoding='utf-8') as f:
        return json.load(f)

def load_debug_tasks_with_models(eval_folder: Path) -> dict[str, dict]:
    """
    Load all debug tasks from eval folder with their task models extracted.
    
    Returns:
        Dict mapping task title to task model dict
    """
    debug_files = get_debug_task_files(eval_folder)
    tasks_by_title = {}
    
    for file in debug_files:
        task_model = extract_task_model_from_debug_file(file)
        if task_model and 'title' in task_model:
            title = task_model['title']
            tasks_by_title[title] = task_model
    
    return tasks_by_title

def create_task_pairs(eval_folder: Path, notion_tasks_file: Path) -> list[tuple[dict, dict]]:
    """
    Create pairs of (notion_task, debug_task) for evaluation.
    
    Returns:
        List of tuples where each tuple is (notion_task_dict, debug_task_model_dict)
    """
    # Load both datasets
    notion_tasks = load_notion_tasks(notion_tasks_file)
    debug_tasks = load_debug_tasks_with_models(eval_folder)
    
    # Create pairs based on matching titles
    pairs = []
    matched_titles = set()
    
    for notion_task in notion_tasks:
        title = notion_task['title']
        if title in debug_tasks:
            pairs.append((notion_task, debug_tasks[title]))
            matched_titles.add(title)
    
    # Report unmatched tasks
    unmatched_debug = set(debug_tasks.keys()) - matched_titles
    unmatched_notion = {t['title'] for t in notion_tasks} - matched_titles
    
    print(f"✓ Created {len(pairs)} task pairs")
    if unmatched_debug:
        print(f"⚠ {len(unmatched_debug)} debug tasks not found in Notion")
        print(f"  Sample unmatched: {list(unmatched_debug)[:3]}")
    if unmatched_notion:
        print(f"⚠ {len(unmatched_notion)} Notion tasks not found in debug")
        print(f"  Sample unmatched: {list(unmatched_notion)[:3]}")
    
    return pairs

# Create task pairs
task_pairs = create_task_pairs(EVAL_FOLDER_PATH, NOTION_TASKS_FILE)
print(f"\n{'='*60}")
print(f"EVALUATION DATASET READY")
print(f"{'='*60}")
print(f"Total pairs: {len(task_pairs)}")
if task_pairs:
    print(f"\nSample pair:")
    print(f"  Notion task: {task_pairs[0][0]['title']}")
    print(f"  Debug task:  {task_pairs[0][1]['title']}")

In [ ]:
# ============ HELPER FUNCTIONS FOR ANALYSIS ============

def inspect_pair(pair_index: int = 0):
    """Display detailed comparison of a task pair."""
    if pair_index >= len(task_pairs):
        print(f"Error: Only {len(task_pairs)} pairs available")
        return
    
    notion_task, debug_task = task_pairs[pair_index]
    
    print(f"{'='*70}")
    print(f"TASK PAIR #{pair_index}: {notion_task['title']}")
    print(f"{'='*70}\n")
    
    print("📊 NOTION TASK (Actual):")
    print(f"  Importance: {notion_task['importance']}")
    print(f"  Urgency:    {notion_task['urgency']}")
    print(f"  Impact:     {notion_task['impact']}")
    print(f"  AI Status:  {notion_task['ai_use_status']}")
    print(f"  AI Useful:  {notion_task['ai_usefulness']}")
    print(f"  Projects:   {len(notion_task['project_ids'])} linked")
    
    print("\n🔍 DEBUG TASK (Expected):")
    print(f"  Importance: {debug_task['importance']}")
    print(f"  Urgency:    {debug_task['urgency']}")
    print(f"  Impact:     {debug_task['impact']}")
    print(f"  AI Status:  {debug_task['ai_use_status']}")
    print(f"  Projects:   {len(debug_task['projects'])} in model")
    print(f"  Confidence: {debug_task.get('confidence', 'N/A')}")
    
    print("\n📝 Original Note:")
    print(f"  {debug_task['original_note'][:200]}...")
    
    if debug_task.get('enrichment'):
        print("\n✨ Enrichment:")
        print(f"  {debug_task['enrichment'][:200]}...")

# Example: Inspect first pair
if task_pairs:
    inspect_pair(0)

In [ ]:
# ============ QUICK ACCESS TO DATA ============

# The main data structure for evaluation:
# task_pairs = [(notion_task, debug_task), ...]
#
# Access pattern:
# for notion_task, debug_task in task_pairs:
#     # Compare notion_task['importance'] with debug_task['importance']
#     # etc.

print("\n📦 Available data structures:")
print(f"  task_pairs:           {len(task_pairs)} pairs of (notion_task, debug_task)")
print(f"  notion_tasks_parsed:  {len(notion_tasks_parsed)} Notion tasks")
print(f"  debug_titles:         {len(debug_titles)} debug task titles")
print(f"\n💾 Files:")
print(f"  {NOTION_TASKS_FILE}")
print(f"\n🔧 Helper functions:")
print(f"  inspect_pair(index) - Detailed view of a task pair")
print(f"  get_debug_task_files() - List all debug task markdown files")
print(f"  load_notion_tasks() - Reload Notion tasks from JSON")

print(f"\n{'='*70}")
print("Ready for evaluation! 🚀")
print(f"{'='*70}")